In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import spacy
import pandas as pd

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
reviews = pd.read_csv('../input/amazon-music-reviews/Musical_instruments_reviews.csv')
reviews.head()

In [ ]:
def get_score(num):
    if num >= 4.0:
        return 2
    elif num==3.0:
        return 1
    else:
        return 0

In [ ]:
new_column = pd.DataFrame({'label': [get_score(review.overall) for idx, review in reviews.iterrows()]})
reviews = reviews.merge(new_column, left_index = True, right_index = True)

In [ ]:
reviews.head()

In [ ]:
reviews.reviewText = reviews.reviewText.astype(str)

In [ ]:
doc_vectors = []
with nlp.disable_pipes():
    for idx, review in reviews.iterrows():
        doc_vectors.append(nlp(review.reviewText).vector)
    
vectors = np.array([doc_vectors])
print(vectors.shape)

In [ ]:
y = np.array(reviews.label.values.tolist())
print(y.shape)

In [ ]:
vectors = vectors.reshape(vectors.shape[1:])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.1, random_state=1)


In [ ]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=1, dual=False, multi_class="ovr", max_iter=10000)

In [ ]:
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

In [ ]:
def predictions(prediction):
    if prediction == 2:
        return "Positive"
    elif prediction == 1:
        return "Neutral"
    else:
        return "Negative"